In [1]:
import tensorflow as tf

In [2]:
s = tf.io.read_file('day10.txt')

@tf.function()
def solve(s):
    s = tf.strings.split(s, '\n')
    joltages = tf.strings.to_number(s, tf.int32)

    joltages = tf.concat(
        ([0], tf.sort(joltages, direction='DESCENDING')),
        axis=0
    )
    joltages = tf.concat(
        (joltages, [joltages[-1]+3]),
        axis=0
    )
    joltage_diffs = joltages[:-1] - joltages[1:]

    ones = tf.where(joltage_diffs == 1)
    threes = tf.where(joltage_diffs == 3)
    return tf.shape(ones)[0] * tf.shape(threes)[0]

%timeit solve(s)

1e+03 µs ± 5.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
s = tf.io.read_file('day10.txt')

@tf.function
def solve(s):
    s = tf.strings.split(s, '\n')
    joltages = tf.strings.to_number(s, tf.float64)

    joltages = tf.concat(
        (tf.sort(joltages, direction='DESCENDING'), [0]),
        axis=0
    )
    joltages = tf.concat(
        ([joltages[0]+3], joltages),
        axis=0
    )
    
    n = tf.shape(joltages, tf.int32)[0]
    i = tf.constant(0, tf.int32)
    
    ta = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
    ta_i = 0
    # We dont need the full matrix, lets make it as dense as possible
    rnge = tf.range(n)
    
    while i < n:
        joltage = joltages[i]
        next_joltages = joltages[i+1:i+4]
        next_joltage_idxs = rnge[i+1:i+4]
        next_joltage_idxs = next_joltage_idxs[(joltage-next_joltages)<=3]
        joltage_idx_repeated = tf.repeat(i, tf.shape(next_joltage_idxs)[0])
        indices = tf.stack((joltage_idx_repeated, next_joltage_idxs), axis=1)
        for j in tf.range(tf.shape(indices)[0]):
            ta = ta.write(ta_i, indices[j])
            ta_i += 1
        i += 1
        
    indices = ta.stack()
    
    amat = tf.scatter_nd(indices, tf.ones(tf.shape(indices)[0], tf.float64), (n, n))

    travel_vec = tf.one_hot(0, tf.cast(n, tf.int32), dtype=tf.float64)[None,:]
    total = tf.constant(0, tf.float64)
    i = 0
    while tf.math.count_nonzero(travel_vec) > 0:
        travel_vec @= amat
        total += travel_vec[0,-1]
        i += 1
    return total

solve(s)

<tf.Tensor: shape=(), dtype=float64, numpy=74049191673856.0>

In [4]:
%timeit solve(s)

52.6 ms ± 677 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
